In [1]:
# Required libraries
import os
from dotenv import load_dotenv, find_dotenv
import base64
import urllib
import requests as rq
import json
import pandas as pd
import time

In [2]:
# Get authentication token
def get_oauth_token():

    url = "https://api.idealista.com/oauth/token"

    load_dotenv(find_dotenv('creds.env')) # Load .env file
    apikey = os.environ.get("API_KEY")
    secret = os.environ.get("SECRET")
    apikey_secret = apikey + ':' + secret

    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1] # Get base64 encoded string

    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params) # Get response
    bearer_token = json.loads(content.text)['access_token'] # Get access token

    return bearer_token

In [3]:
# Get list of properties
def search_api(token, params):
    url = "https://api.idealista.com/3.5/es/search"

    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token} 
    content = rq.post(url, headers=headers, params=params) # Get response
    
    print(content)
    return content

**Note**: *locationId* goes from 0-EU-ES-01 to 0-EU-ES-56 for Spain.

**REQUIREMENTS**:
- country = ['es', 'it', 'pt']
- operation = ['sale', 'rent']
- propertyType = ['homes', 'offices', 'premises', 'garages', 'bedrooms']
- you must specify a center + distance or locationId in each request

In [4]:
params = {
    "country" : 'es',
    "operation" : "rent",
    "propertyType" : "homes",
    "locationId" : "0-EU-ES-46",
    "maxItems" : 50,
}

# Test connection
token = get_oauth_token()
result = search_api(token, params)

<Response [200]>


In [5]:
# Print result
result.text

'{"elementList":[{"propertyCode":"97693036","thumbnail":"https://img3.idealista.com/blur/WEB_LISTING/0/id.pro.es.image.master/cb/2b/e3/987270204.jpg","externalReference":"AV2203006-1","numPhotos":30,"floor":"11","price":1500.0,"propertyType":"penthouse","operation":"rent","size":80.0,"exterior":true,"rooms":3,"bathrooms":2,"address":"Avenida de la Tarongina","province":"València","municipality":"Tavernes de la Valldigna","country":"es","latitude":39.1010767,"longitude":-0.2241471,"showAddress":false,"url":"https://www.idealista.com/inmueble/97693036/","description":"Ático en alquiler en Tavernes de la Valldigna   Ático luminoso en Tavernes de la Valldigna para alquiler vacacional. Tiene tres habitaciones, dos cuartos de baño, terraza muy amplia acristalada (45m2), terraza muy grande solarium (70m2) en la parte superior del apartamento. En el precio estaría incluida una plaza de garaje. La vivienda es un ático, está en una planta 11 con vistas espectaculares, mucha luz y muy ventilado. 

In [6]:
# get totalPages
totalPages = json.loads(result.text)['totalPages']
print(totalPages)

53


In [7]:
# get actual page
actualPage = json.loads(result.text)['actualPage']
print(actualPage)

1


In [8]:
df_tot = pd.DataFrame(json.loads(result.text)["elementList"])
df_tot

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,detailedType,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,district,neighborhood,labels
0,97693036,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2203006-1,30,11,1500.0,penthouse,rent,80.0,True,...,"{'typology': 'flat', 'subTypology': 'penthouse'}","{'subtitle': 'Tavernes de la Valldigna', 'titl...",False,False,False,False,False,NaN,NaN,NaN
1,98039610,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2206084,18,3,1600.0,flat,rent,193.0,True,...,{'typology': 'flat'},"{'subtitle': 'Pinedo', 'title': 'Piso'}",False,False,False,False,False,NaN,NaN,NaN
2,98042660,https://img3.idealista.com/blur/WEB_LISTING/0/...,IM648,9,3,850.0,flat,rent,110.0,True,...,{'typology': 'flat'},"{'subtitle': 'El Cabanyal-El Canyamelar, Valèn...",False,False,False,False,False,Poblats Marítims,El Cabanyal-El Canyamelar,NaN
3,98042528,https://img3.idealista.com/blur/WEB_LISTING/0/...,V1134,29,3,950.0,flat,rent,101.0,True,...,{'typology': 'flat'},"{'subtitle': 'El Carme, València', 'title': 'P...",False,False,True,False,False,Ciutat Vella,El Carme,NaN
4,98042519,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2206086,18,bj,900.0,flat,rent,45.0,True,...,{'typology': 'flat'},"{'subtitle': 'El Pilar, València', 'title': 'P...",False,False,False,False,False,Ciutat Vella,El Pilar,NaN
5,98042463,https://img3.idealista.com/blur/WEB_LISTING/0/...,11100,15,3,1290.0,flat,rent,110.0,True,...,{'typology': 'flat'},"{'subtitle': 'El Pla del Remei, València', 'ti...",False,False,False,False,False,L'Eixample,El Pla del Remei,NaN
6,98042205,https://img3.idealista.com/blur/WEB_LISTING/0/...,p-puig50,47,1,800.0,flat,rent,100.0,True,...,{'typology': 'flat'},"{'subtitle': 'Sotolivar - Mas del Rosari, Pate...",False,False,False,False,False,Sotolivar - Mas del Rosari,NaN,NaN
7,97791775,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,40,2,1700.0,flat,rent,113.0,True,...,{'typology': 'flat'},"{'subtitle': 'La Carrasca, València', 'title':...",False,False,False,False,False,Algirós,La Carrasca,NaN
8,97725193,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,39,1,1700.0,flat,rent,110.0,True,...,{'typology': 'flat'},"{'subtitle': 'L'Illa Perduda, València', 'titl...",False,False,False,False,False,Algirós,L'Illa Perduda,NaN
9,98042144,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,16,4,800.0,flat,rent,71.0,True,...,{'typology': 'flat'},"{'subtitle': 'Nou Moles, València', 'title': '...",False,False,False,False,False,L'Olivereta,Nou Moles,NaN


In [9]:
for i in range(2, totalPages):
    try:
        params['numPage'] = i
        result = search_api(token, params)
        df = pd.DataFrame(json.loads(result.text)["elementList"])
        df_tot = pd.concat([df_tot, df])
        time.sleep(5)
    except Exception as e:
        print(e, "Page: ", i)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [13]:
df_tot.reset_index(drop=True, inplace=True)

In [14]:
# ¿All the propertyCode are unique?
df_tot['propertyCode'].nunique()

2600

In [17]:
df_tot

,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,suggestedTexts,hasPlan,has3DTour,has360,hasStaging,topNewDevelopment,district,neighborhood,labels,newDevelopmentFinished
0,97693036,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2203006-1,30,11,1500.0,penthouse,rent,80.0,True,...,"{'subtitle': 'Tavernes de la Valldigna', 'titl...",False,False,False,False,False,NaN,NaN,NaN,NaN
1,98039610,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2206084,18,3,1600.0,flat,rent,193.0,True,...,"{'subtitle': 'Pinedo', 'title': 'Piso'}",False,False,False,False,False,NaN,NaN,NaN,NaN
2,98042660,https://img3.idealista.com/blur/WEB_LISTING/0/...,IM648,9,3,850.0,flat,rent,110.0,True,...,"{'subtitle': 'El Cabanyal-El Canyamelar, Valèn...",False,False,False,False,False,Poblats Marítims,El Cabanyal-El Canyamelar,NaN,NaN
3,98042528,https://img3.idealista.com/blur/WEB_LISTING/0/...,V1134,29,3,950.0,flat,rent,101.0,True,...,"{'subtitle': 'El Carme, València', 'title': 'P...",False,False,True,False,False,Ciutat Vella,El Carme,NaN,NaN
4,98042519,https://img3.idealista.com/blur/WEB_LISTING/0/...,AV2206086,18,bj,900.0,flat,rent,45.0,True,...,"{'subtitle': 'El Pilar, València', 'title': 'P...",False,False,False,False,False,Ciutat Vella,El Pilar,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,97334169,https://img3.idealista.com/blur/WEB_LISTING/0/...,V-1616,19,1,450.0,flat,rent,86.0,True,...,"{'subtitle': 'Alginet', 'title': 'Piso'}",False,False,False,False,False,NaN,NaN,NaN,NaN
2596,88772862,https://img3.idealista.com/blur/WEB_LISTING/0/...,9853,8,bj,500.0,flat,rent,60.0,True,...,"{'subtitle': 'Bonrepos i Mirambell', 'title': ...",False,False,False,False,False,NaN,NaN,NaN,NaN
2597,94396615,https://img3.idealista.com/blur/WEB_LISTING/0/...,46-205-4752,9,4,600.0,flat,rent,45.0,True,...,"{'subtitle': 'La Petxina, València', 'title': ...",False,False,False,False,False,Extramurs,La Petxina,"[{'name': 'apartamentoType', 'text': 'Apartame...",NaN
2598,98010151,NaN,26133,0,2,560.0,flat,rent,126.0,True,...,"{'subtitle': 'Zona Concordia, Burjassot', 'tit...",False,False,False,False,False,Zona Concordia,NaN,NaN,NaN


**END OF TEST**, next step is to make a file named 'make_data.py' that will generate the data for the analysis.

In [18]:
df_tot.to_csv('../data/rent_Valencia.csv', index=False)